In [1]:
import uuid
import json

import requests

from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
service_load_balancer = 'http://localhost:8000'
session_id = uuid.uuid4().hex
user_phone_number = 'my_did'
request_content = {'auth_id': session_id, 'recipient': user_phone_number}

In [3]:
result = requests.post(service_load_balancer, json=request_content, verify=False)
assert result.status_code == 200
print(result.text)

{"message": "Success: Verification code created.", "auth_id": "ba3bf23aee5440089c4d58d9cd6f9793"}


An equivalent curl request:
~~~
curl -v -X POST -d '{"auth_id": "sha256", "recipient": "12069928996"}' -H "Content-Type: application/json" -k https://sms-proxy-lb-1041651126.us-west-2.elb.amazonaws.com
~~~

Now lets make 4 bad attempts to see the behavior, and then we'll generate another code (currently the invalid attempts threshold is 3, and the expiration window is 1 hour)

In [4]:
for invalid_code in ('1232', '9999', '0000000', '44444'):
    authorization_args = {'auth_id': session_id, 'code': invalid_code}
    response = requests.get(service_load_balancer, params=authorization_args, verify=False)
    print(response.text)

{
  "attempts_left": 2, 
  "message": "Invalid code", 
  "reason": "InvalidAuthCode"
}
{
  "attempts_left": 1, 
  "message": "Invalid code", 
  "reason": "InvalidAuthCode"
}
{
  "attempts_left": 0, 
  "message": "Invalid code", 
  "reason": "InvalidAuthCode"
}
{
  "attempts_left": 0, 
  "message": "Unknown auth id", 
  "reason": "UnknownAuthId"
}


In [ ]:
result = requests.post(service_load_balancer, json=request_content, verify=False)
print(result.text)

In [ ]:
valid_code = '7293'  # Fill me in
authorization_args = {'auth_id': session_id, 'code': valid_code}

In [ ]:
response = requests.get(service_load_balancer, params=authorization_args, verify=False)
print(response.text)